In [1]:
import EEGAnalysis as ea

import numpy as np
import pandas as pd
import h5py
from pprint import pprint
import os, re, json, shutil, random
from hashlib import sha256
from tqdm import tqdm

import matplotlib.pyplot as plt
from IPython import display

In [2]:
_data_dir = './Data'  ## set data folder
ea_manager = ea.DataManager(_data_dir)  ## create data manager

patient_id = 'fansulong'
patient = ea_manager.get_patient(patient_id)  ## load patient

In [3]:
roi = (-2, 5)
tspec = np.linspace(roi[0], roi[1], (roi[1] - roi[0]) * 2000)
frange = np.logspace(np.log10(1), np.log10(150), 30)

In [4]:
for chidx in tqdm(range(len(patient._sgch_config['chidx']))):
    entry = patient.load_isplit(chidx, '20190110-1')

    _epoch = ea.create_1d_epoch_bymarker(entry['20190110-1']['value'], patient.get_marker('20190110-1')[:20], (-2,5), 2000)

    _dwt_result = ea.decomposition.dwt.dwt(data=_epoch, frange=frange, fs=2000, reflection=True)
    _pwr = ea.decomposition.dwt_power(dwtresult=_dwt_result, fs=2000, zscore=True)

    plt.figure(figsize=(8,6))
    plt.contourf(tspec, frange, _pwr, 80)
    plt.clim((-3 * np.std(_pwr), 3 * np.std(_pwr)))
#     plt.colorbar()
#     plt.show()
    plt.tight_layout()
    plt.savefig('export/heatmap/ch_%03d.png'%chidx, bbox_inches='tight')
    plt.close()


100%|██████████| 73/73 [06:46<00:00,  5.86s/it]


In [18]:
loadedf = ea.io.loadedf
detect_cross_pnt = ea.decomposition.detect_cross_pnt

class _DEV(ea.datamanager.Patient):
    def update_DC_marker(self, overwrite=False):
        for item in self._raw_config.values():
            if item['name'] in list(self._marker.file) and not overwrite:
                continue
            elif item['name'] in list(self._marker.file) and overwrite:
                print('overwrite the markers of %s'%(item['name']))
            else:
                pass
            
            _edf = loadedf(item['file'], 'parse marker')
            
            try:
                _grating_marker_ch = np.where([True if item == 'POL DC10' else False for item in _edf.channelLabels])[0][0]
                _clapping_marker_ch = np.where([True if item == 'POL DC09' else False for item in _edf.channelLabels])[0][0]
                print("file %s: DC10: %d"%(item['name'], _grating_marker_ch))
            except IndexError:
                print('file %s has no DC channels'%item['name'])
                continue
            
            _grating_trace = _edf.data[_grating_marker_ch] * _edf.physical_unit[_grating_marker_ch] / 1e6  # unit as Volt
#             _clapping_trace = _edf.data[_clapping_marker_ch] * _edf.physical_unit[_clapping_marker_ch] / 1e6  # unit as Volt
            
            _grating_time = np.array(detect_cross_pnt(_grating_trace, 3, gap=_edf.fs)) / _edf.fs
#             _clapping_time = np.array(detect_cross_pnt(_clapping_trace, 3, gap=_edf.fs)) / _edf.fs
            
            _grating_markers = [{'file':item['name'], 'paradigm':'', 'marker':_i, 'mbias':'0','note':''} for _i in _grating_time]
#             _clapping_markers = [{'file':item['name'], 'paradigm':'', 'marker':_i, 'mbias':'0','note':''} for _i in _clapping_time]
            
            self._marker = self._marker.append(_grating_markers)
            self._update_marker()
            
    def get_marker(self, name, paradigm=None):
        if paradigm == None:
            _temp = self._marker.marker[self._marker.file == name]+\
                self._marker.mbias[self._marker.file == name]
            return _temp.values
        else:
            _temp = self._marker.marker[(self._marker.file == name)&(self._marker.paradigm == name)]+\
                self._marker.mbias[(self._marker.file == name)&(self._marker.paradigm == name)]
            return _temp.values

patient = _DEV('./Data', 'fansulong')



In [22]:
patient.get_marker('20190110-1')

array([ 25.765,  30.817,  35.834,  40.871,  45.889,  50.922,  55.944,
        60.964,  65.984,  71.003,  76.022,  81.041,  86.061,  91.079,
        96.098, 101.136, 106.153, 111.172, 116.192, 121.212, 213.186,
       223.213, 233.234, 243.256, 253.278, 263.3  , 273.32 , 283.344,
       293.365, 303.387, 313.409, 323.43 , 333.451, 343.486, 353.511,
       363.533, 373.555, 383.668, 393.615, 403.637])